In [ ]:
!pip install transformers==4.17

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 68.9 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [ ]:
!pip install transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.9 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

In [ ]:
raw_datasets = load_dataset("squad")
raw_datasets

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [ ]:
raw_datasets['train'][1]['title']

'University_of_Notre_Dame'

In [ ]:
raw_datasets['train'][1]['context']

'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.'

In [ ]:
raw_datasets['train'][1]['question']

'What is in front of the Notre Dame Main Building?'

In [ ]:
raw_datasets['train'][1]['answers']

{'text': ['a copper statue of Christ'], 'answer_start': [188]}

In [ ]:
raw_datasets['train'][1]['id']

'5733be284776f4190066117f'

In [ ]:
raw_datasets['train'].filter(lambda x: len(x['answers']['text']) != 1)

Filter:   0%|          | 0/87599 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 0
})

In [ ]:
raw_datasets['validation'][2]['context']

'Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi\'s Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.'

In [ ]:
raw_datasets['validation'][2]['question']

'Where did Super Bowl 50 take place?'

In [ ]:
raw_datasets['validation'][2]['answers']

{'text': ['Santa Clara, California',
  "Levi's Stadium",
  "Levi's Stadium in the San Francisco Bay Area at Santa Clara, California."],
 'answer_start': [403, 355, 355]}

In [ ]:
raw_datasets['validation'][0]['answers']

{'text': ['Denver Broncos', 'Denver Broncos', 'Denver Broncos'],
 'answer_start': [177, 177, 177]}

In [ ]:
from transformers import AutoTokenizer

In [ ]:
model_checkpoint = 'distilbert-base-cased'

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

#this can handle 2 input texts as we saw in textual entailment task

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/465 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

In [ ]:
context = raw_datasets['train'][1]['context']
question = raw_datasets['train'][1]['question']

inputs = tokenizer(question, context)
tokenizer.decode(inputs['input_ids'])

'[CLS] What is in front of the Notre Dame Main Building? [SEP] Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend " Venite Ad Me Omnes ". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive ( and in a direct line that connects through 3 statues and the Gold Dome ), is a simple, modern stone statue of Mary. [SEP]'

In [ ]:
#context = 'I like eggs'
#question = 'What do you like?'

#inputs = tokenizer(question, context)
#tokenizer.decode(inputs['input_ids'])

In [ ]:
inputs = tokenizer(question, context, max_length=100, truncation = 'only_second', stride=50, return_overflowing_tokens=True)

# max length = 100 includes question, answers, and special tokens
# only truncate the 2nd argument thats the context, but not the question
# stride = 50 means how much overlab between the context windows is when they are split up
# return_overflowing_tokens, is the overlap tokens, when the stride is a positive number

In [ ]:
inputs.keys()

dict_keys(['input_ids', 'attention_mask', 'overflow_to_sample_mapping'])

In [ ]:
inputs['overflow_to_sample_mapping']

[0, 0, 0, 0]

In [ ]:
# If the context is split into multiple windows, we get multiple input samples from one original input sample

In [ ]:
inputs = tokenizer(
    raw_datasets['train'][:3]['question'],
    raw_datasets['train'][:3]['context'],
    max_length=100,
    truncation='only_second',
    stride=50,
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
)

inputs['overflow_to_sample_mapping']

[0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2]

In [ ]:
inputs['offset_mapping']

[[(0, 0),
  (0, 2),
  (3, 7),
  (8, 11),
  (12, 15),
  (16, 22),
  (23, 27),
  (28, 37),
  (38, 44),
  (45, 47),
  (48, 52),
  (53, 55),
  (56, 59),
  (59, 63),
  (64, 70),
  (70, 71),
  (0, 0),
  (0, 13),
  (13, 15),
  (15, 16),
  (17, 20),
  (21, 27),
  (28, 31),
  (32, 33),
  (34, 42),
  (43, 52),
  (52, 53),
  (54, 56),
  (56, 58),
  (59, 62),
  (63, 67),
  (68, 76),
  (76, 77),
  (77, 78),
  (79, 83),
  (84, 88),
  (89, 91),
  (92, 93),
  (94, 100),
  (101, 107),
  (108, 110),
  (111, 114),
  (115, 121),
  (122, 126),
  (126, 127),
  (128, 139),
  (140, 142),
  (143, 148),
  (149, 151),
  (152, 155),
  (156, 160),
  (161, 169),
  (170, 173),
  (174, 180),
  (181, 183),
  (183, 184),
  (185, 187),
  (188, 189),
  (190, 196),
  (197, 203),
  (204, 206),
  (207, 213),
  (214, 218),
  (219, 223),
  (224, 226),
  (226, 229),
  (229, 232),
  (233, 237),
  (238, 241),
  (242, 248),
  (249, 250),
  (250, 251),
  (251, 254),
  (254, 256),
  (257, 259),
  (260, 262),
  (263, 264),
  (264, 2

In [ ]:
for ids in inputs['input_ids']:
  print(tokenizer.decode(ids))

[CLS] To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France? [SEP] Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend " Venite Ad Me Omnes ". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basi [SEP]
[CLS] To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France? [SEP] the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend " Venite Ad Me Omnes ". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin [SEP]
[CLS] To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France? [SEP] Next to the Main Building is the B

In [ ]:
inputs = tokenizer(
    question,
    context,
    max_length=100,
    truncation='only_second',
    stride=50,
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
)

inputs.keys()

dict_keys(['input_ids', 'attention_mask', 'offset_mapping', 'overflow_to_sample_mapping'])

In [ ]:
inputs['offset_mapping']

[[(0, 0),
  (0, 4),
  (5, 7),
  (8, 10),
  (11, 16),
  (17, 19),
  (20, 23),
  (24, 29),
  (30, 34),
  (35, 39),
  (40, 48),
  (48, 49),
  (0, 0),
  (0, 13),
  (13, 15),
  (15, 16),
  (17, 20),
  (21, 27),
  (28, 31),
  (32, 33),
  (34, 42),
  (43, 52),
  (52, 53),
  (54, 56),
  (56, 58),
  (59, 62),
  (63, 67),
  (68, 76),
  (76, 77),
  (77, 78),
  (79, 83),
  (84, 88),
  (89, 91),
  (92, 93),
  (94, 100),
  (101, 107),
  (108, 110),
  (111, 114),
  (115, 121),
  (122, 126),
  (126, 127),
  (128, 139),
  (140, 142),
  (143, 148),
  (149, 151),
  (152, 155),
  (156, 160),
  (161, 169),
  (170, 173),
  (174, 180),
  (181, 183),
  (183, 184),
  (185, 187),
  (188, 189),
  (190, 196),
  (197, 203),
  (204, 206),
  (207, 213),
  (214, 218),
  (219, 223),
  (224, 226),
  (226, 229),
  (229, 232),
  (233, 237),
  (238, 241),
  (242, 248),
  (249, 250),
  (250, 251),
  (251, 254),
  (254, 256),
  (257, 259),
  (260, 262),
  (263, 264),
  (264, 265),
  (265, 268),
  (268, 269),
  (269, 270),
 

In [ ]:
len(inputs['offset_mapping'])

4

In [ ]:
len(inputs['offset_mapping'][1])

#because truncation is 100

100

In [ ]:
print(inputs.sequence_ids(0))

# it is like a remenescent of token type ids we saw earlier
# we cannot use token type ids, because it is not present in all models such as distilbert
# This is the list that has the same length as the tokenized input
# 0's denote which are the part of the question and 1's denote tokens which are te parts of the context


[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, None]


In [ ]:
answer = raw_datasets['train'][1]['answers']
answer

# problem: the postion of the answer will change in each window of the context
# answer is the target of the neural network

# the representation of the answer won't work because it is relative to the full context
# the answer is represented by the character position, but not the token position, as required by the neural network
#

{'text': ['a copper statue of Christ'], 'answer_start': [188]}

In [ ]:
type(inputs.sequence_ids(0))

list

In [ ]:
# find the start and end of the context (first and last 1)

sequence_ids = inputs.sequence_ids(0) # returns a list

ctx_start = sequence_ids.index(1) # first 1
ctx_end = len(sequence_ids) - sequence_ids[::-1].index(1) - 1  #last 1

ctx_start, ctx_end

#index() function says the first occurence of 1
#ctx_end is 98, makes sense because the token at 99 is [SEP]

(13, 98)

In [ ]:
#ctx_start = sequence_ids.index(1) # first 1
#ctx_end = len(sequence_ids) - sequence_ids[::-1].index(1) - 1  #last 1

In [ ]:
# we are going from the character position to the token postion

# check whether or not the answer is fully contained within the context
# if not, set target as (start, end) = (0,0)

ans_start_char = answer['answer_start'][0]
ans_end_char = ans_start_char + len(answer['text'][0])

offset = inputs['offset_mapping'][0]  # we are currently looking at the first input sample

start_idx = 0
end_idx = 0

if offset[ctx_start][0] > ans_start_char or offset[ctx_end][1] < ans_end_char:
  print("target is (0,0)")

else:
  i = ctx_start
  for start_end_char in offset[ctx_start:]:
    start, end = start_end_char

    if start == ans_start_char:
      start_idx = i
    if end == ans_end_char:
      end_idx = i
      break

    i += 1


# the more cleaner way to do this is
# if ( ans within context ):
#     find start_idx, end_idx

start_idx, end_idx

(53, 57)

In [ ]:
# offset[ctx_start][0] will give the start character position of the first token in the context
# offset[ctx_end][0] will give the end character position of the first token in the context

In [ ]:
# if offset[ctx_start][0] > ans_start_char or off[ctx_end][0] < ans_end_char
#   print("target is (0,0) ")
# else:
#   Find start and end token postions

In [ ]:
#i = ctx_start

#for start_end_index in offset[ctx_start:]:
#  start, end = start_end_index
#  if start == ans_start_char:
#    start_idx = i
#  if end == ans_end_char:
#    end_idx = i
#    break

#  i + = 1

In [ ]:
# check

input_ids = inputs['input_ids'][0]
input_ids[start_idx : end_idx+1]

[170, 7335, 5921, 1104, 4028]

In [ ]:
tokenizer.decode(input_ids[start_idx : end_idx+1])

'a copper statue of Christ'

In [ ]:
def find_answer_token_idx(
    ctx_start,
    ctx_end,
    ans_start_char,
    ans_end_char,
    offset):

  start_idx = 0
  end_idx = 0

  if offset[ctx_start][0] > ans_start_char or offset[ctx_end][1] < ans_end_char:
    pass
  else:
    i = ctx_start
    for start_end_char in offset[ctx_start:]:
      start, end = start_end_char

      if start == ans_start_char:
        start_idx = i
      if end == ans_end_char:
        end_idx = i
        break

      i += 1

  return start_idx, end_idx

In [ ]:
start_idxs = []
end_idxs = []

for i, offset in enumerate(inputs['offset_mapping']):
  sequence_ids = inputs.sequence_ids(i)

  ctx_start = sequence_ids.index(1)
  ctx_end = len(sequence_ids) - sequence_ids[::-1].index(1) - 1

  start_idx, end_idx = find_answer_token_idx(ctx_start, ctx_end, ans_start_char, ans_end_char, offset)

  start_idxs.append(start_idx)
  end_idxs.append(end_idx)

start_idxs, end_idxs

([53, 17, 0, 0], [57, 21, 0, 0])

In [ ]:
for q in raw_datasets['train']['question'][:1000]:
  if q.strip() != q:
    print(q)

In what city and state did Beyonce  grow up? 
 The album, Dangerously in Love  achieved what spot on the Billboard Top 100 chart?
Which song did Beyonce sing at the first couple's inaugural ball? 
What event did Beyoncé perform at one month after Obama's inauguration? 
Where was the album released? 
What movie influenced Beyonce towards empowerment themes? 


In [ ]:
# Goggle used 384 for SQUAD

max_length = 384
stride = 128

def tokenize_fn_train(batch):
  # some questions have leading or trailing whitespace
  questions = [q.strip() for q in batch['question']]

  inputs = tokenizer(
      questions,
      batch['context'],
      max_length = max_length,
      truncation = 'only_second',
      stride = stride,
      return_overflowing_tokens = True,
      return_offsets_mapping = True,
      padding="max_length"

  )

  offset_mapping = inputs.pop('offset_mapping')
  orig_sample_idxs = inputs.pop('overflow_to_sample_mapping')
  answers = batch['answers']
  start_idxs, end_idxs = [],[]

  for i, offset in enumerate(offset_mapping):
    sample_idx = orig_sample_idxs[i]
    answer = answers[sample_idx]

    ans_start_char = answer['answer_start'][0]
    ans_end_char = ans_start_char + len(answer['text'][0])

    sequence_ids = inputs.sequence_ids(i)

    ctx_start = sequence_ids.index(1)
    ctx_end = len(sequence_ids) - sequence_ids[::-1].index(1) - 1

    start_idx, end_idx = find_answer_token_idx(ctx_start, ctx_end, ans_start_char, ans_end_char, offset)

    start_idxs.append(start_idx)
    end_idxs.append(end_idx)

  inputs['start_positions'] = start_idxs
  inputs['end_positions'] = end_idxs

  return inputs


In [ ]:
train_dataset = raw_datasets['train'].map(tokenize_fn_train, batched=True, remove_columns=raw_datasets['train'].column_names)

len(raw_datasets['train']), len(train_dataset)

Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

(87599, 88729)

In [ ]:
raw_datasets['validation'][0]

{'id': '56be4db0acb8001400a502ec',
 'title': 'Super_Bowl_50',
 'context': 'Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi\'s Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.',
 'question': 'Which NFL team represented the AFC at Super Bowl 50?',
 'answers': {'text': ['Denver Broncos', 'Denver Broncos', 'Denver Broncos'],


In [ ]:
# for validation tokenize function 2 things to be noted
# we don't need to generate targets
# we are going to modify offset mapping to some useful information

def tokenize_fn_validation(batch):
  # some questions have leading or trailing whitespace
  questions = [q.strip() for q in batch['question']]

  inputs = tokenizer(
      questions,
      batch['context'],
      max_length = max_length,
      truncation = 'only_second',
      stride = stride,
      return_overflowing_tokens = True,
      return_offsets_mapping = True,
      padding="max_length"

  )

  orig_sample_idxs = inputs.pop('overflow_to_sample_mapping')
  sample_ids = []

  for i in range(len(inputs['input_ids'])):
    sample_idx = orig_sample_idxs[i]
    sample_ids.append(batch['id'][sample_idx])

    sequence_ids = inputs.sequence_ids(i)
    offset = inputs['offset_mapping'][i]
    inputs['offset_mapping'][i] = [
        x if sequence_ids[j] ==1 else None for j,x in enumerate(offset)]  # replacing all non context tokens with None

  inputs['sample_id'] = sample_ids

  return inputs

In [ ]:
validation_dataset = raw_datasets['validation'].map(tokenize_fn_validation, batched=True, remove_columns=raw_datasets['validation'].column_names)

len(raw_datasets['validation']), len(validation_dataset)

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

(10570, 10822)

# **Metric**

In [ ]:
from datasets import load_metric

metric = load_metric("squad")

<ipython-input-49-0eb82bc40e87>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("squad")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for squad contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/squad/squad.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [ ]:
predicted_answers = [
    {'id': 1, 'prediction_text': 'Albert Einstein'},
    {'id': 2, 'prediction_text': 'Physicist'},
    {'id': 3, 'prediction_text': 'general relativity'}
]

true_answers = [
    {'id':1, 'answers' :{'text' : ['Albert Einstein'], 'answer_start' :[100]}},
    {'id':2, 'answers' :{'text': ['physicist'], 'answer_start' :[100]}},
    {'id':3, 'answers' :{'text' :['special relativity'], 'answer_start' : [100]}}
]

metric.compute(predictions=predicted_answers, references=true_answers)

{'exact_match': 66.66666666666667, 'f1': 83.33333333333333}

In [ ]:
small_validation_dataset = raw_datasets['validation'].select(range(100))

In [ ]:
trained_checkpoint = 'distilbert-base-cased-distilled-squad'

In [ ]:
tokenizer2 = AutoTokenizer.from_pretrained(trained_checkpoint)

old_tokenizer = tokenizer

tokenizer = tokenizer2

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/473 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

In [ ]:
small_validation_processed = small_validation_dataset.map(tokenize_fn_validation, batched=True, remove_columns=raw_datasets['validation'].column_names)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
tokenizer = old_tokenizer

In [ ]:
import torch

from transformers import AutoModelForQuestionAnswering

In [ ]:
small_model_inputs = small_validation_processed.remove_columns(['sample_id', 'offset_mapping'])
small_model_inputs.set_format('torch')

In [ ]:
#get gpu device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# move tensors to gpu device
small_model_inputs_gpu = {
    k: small_model_inputs[k].to(device) for k in small_model_inputs.column_names
}

In [ ]:
# download the model

trained_model = AutoModelForQuestionAnswering.from_pretrained(trained_checkpoint).to(device)

Downloading:   0%|          | 0.00/249M [00:00<?, ?B/s]

In [ ]:
#get the model outputs

with torch.no_grad():  # since we won't be using the gradients
  outputs = trained_model(**small_model_inputs_gpu)


outputs
# both start_logits and end_logits have shape NxT
# N=# of samples, T= # of timesteps

QuestionAnsweringModelOutput(loss=None, start_logits=tensor([[ -2.2607,  -5.1783,  -5.2709,  ...,  -9.5243,  -9.5183,  -9.5288],
        [ -2.5961,  -5.5482,  -5.5313,  ...,  -9.9598,  -9.9533,  -9.9860],
        [ -3.7127,  -7.1848,  -8.5388,  ..., -11.6557, -11.6571, -11.6505],
        ...,
        [ -2.0260,  -4.4167,  -4.4980,  ...,  -8.1479,  -8.1530,  -8.1760],
        [ -4.1553,  -5.8304,  -7.1643,  ..., -10.5255, -10.5251, -10.4890],
        [ -3.2000,  -5.8162,  -6.7249,  ...,  -9.4935,  -9.5038,  -9.4871]],
       device='cuda:0'), end_logits=tensor([[ -0.7353,  -4.9236,  -5.1048,  ...,  -8.8734,  -8.8916,  -8.8550],
        [ -1.3056,  -5.3870,  -5.4945,  ...,  -9.4895,  -9.5039,  -9.4958],
        [ -2.7649,  -7.2201,  -9.0916,  ..., -11.3106, -11.3414, -11.2702],
        ...,
        [ -0.0768,  -4.8210,  -4.4374,  ...,  -8.0483,  -8.0502,  -7.9903],
        [ -2.7347,  -5.3650,  -7.2549,  ..., -10.0498, -10.0661,  -9.9886],
        [ -1.0991,  -4.2569,  -6.1267,  ...,  -8

In [ ]:
start_logits = outputs.start_logits.cpu().numpy()
end_logits = outputs.end_logits.cpu().numpy()

# in pytorch, we can get numpy arrays from pytorch tensors by first moving them back to the cpu and then calling the numpy method

In [ ]:
small_validation_processed['sample_id'][:5]

#unique_string identifiers

['56be4db0acb8001400a502ec',
 '56be4db0acb8001400a502ed',
 '56be4db0acb8001400a502ee',
 '56be4db0acb8001400a502ef',
 '56be4db0acb8001400a502f0']

In [ ]:
len(validation_dataset['sample_id'])

10822

In [ ]:
len(set(validation_dataset['sample_id']))

10570

In [ ]:
sample_id2idxs = {}

for i, id_ in enumerate(small_validation_processed['sample_id']):
  if id_ not in sample_id2idxs:
    sample_id2idxs[id_] = [i]
  else:
    print('here')
    sample_id2idxs[id_].append(i)


# ***Converting logits to answers***

In [ ]:
start_logits.shape, end_logits.shape

((100, 384), (100, 384))

In [ ]:
(-start_logits[0]).argsort()

array([ 46,  57,  47,  38,  39,  58,  50,  43,  45,  54,  56,  49,  13,
        42,  40,  35,  27,  31,  48,  41,  53,  44,  37,  59,  78,  15,
         0,  52,  24,  65,  81,  70,  18,  51,  55,  26,  69,  29,  28,
        75,  61,  64,  23,  36,  32,  11, 101,  62,  66,  34,  95,  30,
        63,  21,  19,  20,  17,  14,  22,  33,  68,  87, 171,  12,  76,
        71,  73,  92, 110,  84, 151,   1,  74,   2,   6,  16,  80,  79,
       105,  98,  10,  96, 136, 169, 106, 100,  93, 165,  67, 109,   8,
        90,   3, 115,  60,   5,  97,   7, 103, 102,  86,  72, 111,  89,
       108,   4,  88,  25, 132,  77, 123, 150, 124, 153,  83, 118,  82,
        85, 107, 114, 143, 164, 137, 130, 166, 159, 131,  91,   9, 144,
       139, 160,  94, 141, 128, 112, 134, 152, 170, 154, 117, 127, 104,
       140, 157, 155, 133, 145, 119, 162, 138, 135, 156, 167, 168, 126,
       148, 163, 161, 116,  99, 120, 142, 158, 125, 146, 113, 121, 147,
       149, 129, 122, 311, 312, 304, 309, 313, 310, 300, 307, 31

In [ ]:
n_largest = 20
max_answer_length = 30
predicted_answers = []

In [ ]:
# we are looping through untokenized original dataset
for sample in small_validation_dataset:
  sample_id = sample['id']
  context = sample['context']

  #update these as we loop through the candidate answers
  best_score = float('-inf')
  best_answer = None

  # we will skip to computing answers itself
  # Now loop through the *expanded* input samples
  for idx in sample_id2idxs[sample_id]:
    start_logit = start_logits[idx] # start and end logits are NxT, but this will give size T vector, 384 vectors
    end_logit = end_logits[idx] # 384 vectors
    offsets = small_validation_processed[idx]['offset_mapping']


    #we want to keep None for the question offset mapping, leave as it is context
    #this will be an easy way to determine whether we are in context
    start_indices = (-start_logit).argsort() # argsort() sorts is ascending order by default, so we are negating the list
    end_indices = (-end_logit).argsort() #since we want the largest logit at first, we are arranging in descending order

    for start_idx in start_indices[:n_largest]:
      for end_idx in end_indices[:n_largest]:
        # we will check for invalid answers
        if offsets[start_idx] is None or offsets[end_idx] is None:
          continue

        if end_idx < start_idx:
          continue

        if end_idx-start_idx+1 > max_answer_length:
          continue


        score = start_logit[start_idx] + end_logit[end_idx]

        if best_score < score:
          best_score = score

          first_ch = offsets[start_idx][0]
          last_ch = offsets[end_idx][1]

          best_answer = context[first_ch:last_ch]


  predicted_answers.append({'id': sample_id, 'prediction_text': best_answer})


In [ ]:
small_validation_dataset['answers'][0]

{'text': ['Denver Broncos', 'Denver Broncos', 'Denver Broncos'],
 'answer_start': [177, 177, 177]}

In [ ]:
true_answers = [
    {'id': x['id'], 'answers':x['answers']} for x in small_validation_dataset
]

metric.compute(predictions=predicted_answers, references=true_answers)

{'exact_match': 83.0, 'f1': 88.25000000000004}

In [ ]:
from tqdm.autonotebook import tqdm

def compute_metrics(
    start_logits,
    end_logits,
    processed_dataset,
    orig_dataset):

  sample_id2idxs = {}

  for i, id_ in enumerate(processed_dataset['sample_id']):
    if i not in sample_id2idxs:
      sample_id2idxs[id_] = [i]
    else:
      sample_id2idxs[id_].append(i)

  predicted_answers = []
  for sample in tqdm(orig_dataset):
    sample_id = sample['id']
    context = sample['context']

    best_score = float('-inf')
    best_answer = None

    # now loop through expanded input sample
    for idx in sample_id2idxs[sample_id]:
      start_logit = start_logits[idx]
      end_logit = end_logits[idx]

      # do not do the reverse
      offsets = processed_dataset[idx]['offset_mapping']

      start_indices = (-start_logit).argsort()
      end_indices = (-end_logit).argsort()

      for start_idx in start_indices[:n_largest]:
        for end_idx in end_indices[:n_largest]:
          if offsets[start_idx] is None or offsets[end_idx] is None:
            continue

          if end_idx < start_idx:
            continue

          if end_idx - start_idx +1 > max_answer_length:
            continue

          score = start_logit[start_idx] + end_logit[end_idx]
          if score > best_score:
            best_score = score

            first_ch = offsets[start_idx][0]
            last_ch = offsets[end_idx][1]

            best_answer = context[first_ch:last_ch]


    predicted_answers.append({'id':sample_id, 'prediction_text':best_answer})

  true_answers = [
      {'id':x['id'], 'answers':x['answers']} for x in orig_dataset
  ]

  return metric.compute(predictions=predicted_answers, references=true_answers)


In [ ]:
compute_metrics(start_logits, end_logits, small_validation_processed, small_validation_dataset)

  0%|          | 0/100 [00:00<?, ?it/s]

{'exact_match': 83.0, 'f1': 88.25000000000004}

In [ ]:
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/251M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForQuestionAnswering: ['vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on

In [ ]:
from transformers import TrainingArguments

In [ ]:
args = TrainingArguments(
    'finetuned-squad',
    evaluation_strategy = 'no',
    save_strategy = 'epoch',
    learning_rate = 2e-5,
    num_train_epochs = 3,
    weight_decay=0.01,
    fp16=True
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model = model,
    args = args,
    train_dataset = train_dataset,
    eval_dataset = validation_dataset,
    tokenizer = tokenizer
)

trainer.train()

Using amp half precision backend
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 88729
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 33276


Step,Training Loss
500,3.319400
1000,2.244400
1500,1.990000
2000,1.781700
2500,1.615500
3000,1.514300
3500,1.518000
4000,1.465900
4500,1.452000
5000,1.345100


Saving model checkpoint to finetuned-squad/checkpoint-11092
Configuration saved in finetuned-squad/checkpoint-11092/config.json
Model weights saved in finetuned-squad/checkpoint-11092/pytorch_model.bin
tokenizer config file saved in finetuned-squad/checkpoint-11092/tokenizer_config.json
Special tokens file saved in finetuned-squad/checkpoint-11092/special_tokens_map.json
Saving model checkpoint to finetuned-squad/checkpoint-22184
Configuration saved in finetuned-squad/checkpoint-22184/config.json
Model weights saved in finetuned-squad/checkpoint-22184/pytorch_model.bin
tokenizer config file saved in finetuned-squad/checkpoint-22184/tokenizer_config.json
Special tokens file saved in finetuned-squad/checkpoint-22184/special_tokens_map.json
Saving model checkpoint to finetuned-squad/checkpoint-33276
Configuration saved in finetuned-squad/checkpoint-33276/config.json
Model weights saved in finetuned-squad/checkpoint-33276/pytorch_model.bin
tokenizer config file saved in finetuned-squad/che

TrainOutput(global_step=33276, training_loss=1.0455268156443926, metrics={'train_runtime': 4076.867, 'train_samples_per_second': 65.292, 'train_steps_per_second': 8.162, 'total_flos': 2.608361755366349e+16, 'train_loss': 1.0455268156443926, 'epoch': 3.0})

In [ ]:
trainer_output = trainer.predict(validation_dataset)

type(trainer_output)
predictions,_,_ = trainer_output
start_logits, end_logits = predictions

In [ ]:
compute_metrics(
    start_logits,
    end_logits,
    validation_dataset,
    raw_datasets['validation']
)

In [ ]:
trainer.save_model('my_saved_model')

from transformers import pipeline

In [ ]:
qa = pipeline('question-answering', model='my_saved_model', device=0)

context = 'Today I went to the store to purchase a carton of milk'
question = 'What did I buy?'

qa(context=context, question=question)

NameError: name 'pipeline' is not defined